In [ ]:
import numpy as np
from scipy.interpolate import Rbf
import plotly.plotly as py
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
from mu2e.dataframeprod import DataFrameMaker
from mu2e.src.fiteval_c2 import FitFunctionMaker2
from interp_studies import interp_phi, interp_phi_quad, interp_phi_cubic
from mu2e import mu2e_ext_path
import tqdm

In [ ]:
df = DataFrameMaker(mu2e_ext_path+'datafiles/Mau10/Standard_Maps/Mu2e_DSMap',use_pickle = True).data_frame
ffm = FitFunctionMaker2('../mu2e/src/Mau10_800mm_long.csv')
init_notebook_mode()

In [ ]:
z = 6521
y = 75
#y = 85
#z = 6531

xs = np.linspace(0,700,701)
buffer = 100 # for rbf
df_subset = df.query('{0}<=X<={1} and {2}<=Y<={3} and {4}<=Z<={5}'.format(
    xs[0]-100, xs[-1]+100, y-100, y+100, z-100, z+100))

In [ ]:
bx_rbf = []
by_rbf = []
bz_rbf = []

bx_lacey = []
by_lacey = []
bz_lacey = []

bx_lacey_0 = []
by_lacey_0 = []
bz_lacey_0 = []

bx_lacey_m1 = []
by_lacey_m1 = []
bz_lacey_m1 = []

bx_lacey_1 = []
by_lacey_1 = []
bz_lacey_1 = []

bx_brian = []
by_brian = []
bz_brian = []

bx_map = []
by_map = []
bz_map = []

for x in tqdm.tqdm_notebook(xs):
    # RBF mode
    df_tmp = df_subset.query('{0}<=X<={1}'.format(x-100,x+100))
    rbf_bx = Rbf(df_tmp.X, df_tmp.Y, df_tmp.Z, df_tmp.Bx, function='linear')
    rbf_by = Rbf(df_tmp.X, df_tmp.Y, df_tmp.Z, df_tmp.By, function='linear')
    rbf_bz = Rbf(df_tmp.X, df_tmp.Y, df_tmp.Z, df_tmp.Bz, function='linear')
    bx_rbf.append(float(rbf_bx(x,y,z)))
    by_rbf.append(float(rbf_by(x,y,z)))
    bz_rbf.append(float(rbf_bz(x,y,z)))
    
    # Lacey mode
    _, b_lacey_0 = interp_phi_cubic(df_tmp, x, y, z, plot=False, mode='lacey')
    _, b_lacey_1 = interp_phi_cubic(df_tmp, x, y, z, plot=False, mode='lacey', shift=1)
    _, b_lacey_m1 = interp_phi_cubic(df_tmp, x, y, z, plot=False, mode='lacey', shift=-1)

    # weighting of the form:
    # ((low+high)/2-mid)*x^2 + ((high-low)/2)*x + mid
    x_distance = (x%25)/25-0.5
    wm1 = 1/(abs(x_distance-(-1)))
    w0 = 5/(abs(x_distance-(0)))
    w1 = 1/(abs(x_distance-(1)))
    #bx_lacey.append((b_lacey_0[0]+b_lacey_1[0]+b_lacey_m1[0])/3.0)
    #bx_weight = ((b_lacey_m1[0]+b_lacey_1[0])/2.0-b_lacey_0[0])*x_weight**2 + ((b_lacey_m1[0]-b_lacey_1[0])/2.0)*x_weight+b_lacey_0[0]
    #bx_weight = b_lacey_0[0] + (x_weight-0.5)*b_lacey_1[0] + (x_weight+0.5)*b_lacey_m1[0]
    bx_weight = (wm1*b_lacey_1[0]+w0*b_lacey_0[0]+w1*b_lacey_m1[0])/(wm1+w1+w0)
    bx_lacey.append(bx_weight)
    by_lacey.append(b_lacey[1])
    bz_lacey.append(b_lacey[2])
    bx_lacey_0.append(b_lacey_0[0])
    bx_lacey_m1.append(b_lacey_m1[0])
    bx_lacey_1.append(b_lacey_1[0])

    # Brian mode
    # _, b_brian = interp_phi_cubic(df_tmp, x, y, z, plot=False, mode='brian')
    # bx_brian.append(b_brian[0])
    # by_brian.append(b_brian[1])
    # bz_brian.append(b_brian[2])
    
    # Field Map mode
    b_map = ffm.mag_field_function(x,y,z, True)
    bx_map.append(b_map[0])
    by_map.append(b_map[1])
    bz_map.append(b_map[2])

In [ ]:
layout = dict(title = 'Bx v X, Y={0}, Z={1}'.format(y,z),
              xaxis = dict(title = 'X (mm)', range = [640, 700]),
              yaxis = dict(title = 'Bx (T)', range = [0.0685, 0.069]),
              showlegend=True,
              )
trace_map = go.Scatter(
    x = xs,
    y = bx_map,
    name = 'fit map'
)
trace_rbf = go.Scatter(
    x = xs,
    y = bx_rbf,
    name = 'rbf'
)
trace_lacey = go.Scatter(
    x = xs,
    y = bx_lacey,
    name = 'lacey'
)
trace_l0 = go.Scatter(
    x = xs,
    y = bx_lacey_0,
    name = 'lacey 0'
)
trace_l1 = go.Scatter(
    x = xs,
    y = bx_lacey_1,
    name = 'lacey 1'
)
trace_lm1 = go.Scatter(
    x = xs,
    y = bx_lacey_m1,
    name = 'lacey -1'
)
data = [trace_map, trace_rbf, trace_lacey, trace_l0, trace_l1, trace_lm1]
fig = dict(data=data, layout=layout)
iplot(fig)

In [ ]:
layout = dict(title = 'By v X, Y={0}, Z={1}, Zoom'.format(y,z),
              xaxis = dict(title = 'X (mm)', range=[170,270]),
              yaxis = dict(title = 'By (T)', range=[0.0101065, 0.0101095]),
              showlegend=True,
              )
trace_map = go.Scatter(
    x = xs,
    y = by_map,
    name = 'fit map'
)
trace_rbf = go.Scatter(
    x = xs,
    y = by_rbf,
    name = 'rbf'
)
trace_lacey = go.Scatter(
    x = xs,
    y = by_lacey,
    name = 'lacey'
)
trace_brian = go.Scatter(
    x = xs,
    y = by_brian,
    name = 'lacey hybrid'
)
data = [trace_map, trace_rbf, trace_lacey, trace_brian]
fig = dict(data=data, layout=layout)
iplot(fig)

In [ ]:
layout = dict(title = 'Bz v X, Y={0}, Z={1}'.format(y,z),
              xaxis = dict(title = 'X (mm)'),
              yaxis = dict(title = 'Bz (T)'),
              showlegend=True,
              )
trace_map = go.Scatter(
    x = xs,
    y = bz_map,
    name = 'fit map'
)
trace_rbf = go.Scatter(
    x = xs,
    y = bz_rbf,
    name = 'rbf'
)
trace_lacey = go.Scatter(
    x = xs,
    y = bz_lacey,
    name = 'lacey'
)
trace_brian = go.Scatter(
    x = xs,
    y = bz_brian,
    name = 'lacey hybrid'
)
data = [trace_map, trace_rbf, trace_lacey, trace_brian]
fig = dict(data=data, layout=layout)
iplot(fig)